# Web Scrape Box Office Mojo
## Import Modules

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [2]:
url = 'https://www.boxofficemojo.com/chart/top_lifetime_gross/?ref_=bo_lnav_hm_shrt'
response = requests.get(url)
response.status_code

200

In [3]:
response.text[:1000]

'<!doctype html><html class="a-no-js" data-19ax5a9jf="dingo"><head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/><meta name="viewport" content="width=device-width, initial-scale=1.0" />\n                <meta charset="utf-8" />\n            <title dir="ltr">Top Lifetime Grosses - Box Office Mojo</title><meta content="Top Lifetime Grosses" name="title" />\n            <meta content="Box Office Mojo" property="og:site_name" />\n            <meta content="https://m.media-amazon.com/images/G/01/boxofficemojo/logo/mojo-logo-bg.png" property="og:image"/>\n            <meta name="format-detection" content="telephone=no" />\n            <link href="https://m.media-amazon.com/images/G/01/boxofficemojo/v2/favicon._CB448965889_.ico" type="image/x-icon" rel="icon" />\n            <link rel="stylesheet" href="https://images-na.ssl-images-amazon.com/images/I/11EIQ5IGqaL._RC|012LjolmrML.css,41DAFIecsVL.css,51IB+wfP8qL.css,01ZfXnjPmmL.css,01oDR3IULNL.css,01Vctty9pOL.c

In [4]:
page = response.text
soup = BeautifulSoup(page, 'lxml')
table = soup.find('table')
#table

In [5]:
rows = [row for row in table.find_all('tr')] 

rows[1]

<tr><td class="a-text-right mojo-header-column mojo-truncate mojo-field-type-rank">1</td><td class="a-text-left mojo-field-type-title"><a class="a-link-normal" href="/title/tt2488496/?ref_=bo_cso_table_1">Star Wars: Episode VII - The Force Awakens</a></td><td class="a-text-right mojo-field-type-money">$936,662,225</td><td class="a-text-left mojo-field-type-year"><a class="a-link-normal" href="/year/2015/?ref_=bo_cso_table_1">2015</a></td></tr>

## Collect All Movies
Begin by scraping the 1000 rows of the top grossing movies

In [6]:
url_list = ['https://www.boxofficemojo.com/chart/top_lifetime_gross/?ref_=bo_lnav_hm_shrt', 
            'https://www.boxofficemojo.com/chart/top_lifetime_gross/?offset=200', 
            'https://www.boxofficemojo.com/chart/top_lifetime_gross/?offset=400', 
            'https://www.boxofficemojo.com/chart/top_lifetime_gross/?offset=600', 
            'https://www.boxofficemojo.com/chart/top_lifetime_gross/?offset=800']

rows = []

for url in url_list:
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page,"lxml")
    table = soup.find('table')
    rows.extend(table.find_all('tr')[1:]) 

In [7]:
print(len(rows))

1000


In [8]:
movies = {}

for row in rows[:1001]:
    items = row.find_all('td')
    title_link = items[1].find('a')
    title, url = title_link.text, title_link['href']
    link_stub = title_link['href']
    movies[link_stub] = [url] + [i.text for i in items]
    
print(len(movies))

1000


### Create dataframe 
Convert dictionary to dataframe. Create columns for collected data.

In [9]:
import pandas as pd

top_movies = pd.DataFrame(movies).T  #transpose
top_movies.columns = ['link_stub', 'rank', 'title','lifetime_gross', 'year']

print(len(top_movies))

1000


In [10]:
top_movies.head()

,link_stub,rank,title,lifetime_gross,year
/title/tt2488496/?ref_=bo_cso_table_1,/title/tt2488496/?ref_=bo_cso_table_1,1,Star Wars: Episode VII - The Force Awakens,"$936,662,225",2015
/title/tt4154796/?ref_=bo_cso_table_2,/title/tt4154796/?ref_=bo_cso_table_2,2,Avengers: Endgame,"$858,373,000",2019
/title/tt0499549/?ref_=bo_cso_table_3,/title/tt0499549/?ref_=bo_cso_table_3,3,Avatar,"$760,507,625",2009
/title/tt1825683/?ref_=bo_cso_table_4,/title/tt1825683/?ref_=bo_cso_table_4,4,Black Panther,"$700,426,566",2018
/title/tt4154756/?ref_=bo_cso_table_5,/title/tt4154756/?ref_=bo_cso_table_5,5,Avengers: Infinity War,"$678,815,482",2018


### Create functions
Creating functions to further scrape tables for each movie title.

In [11]:
url = 'https://www.boxofficemojo.com/title/tt2488496/?ref_=bo_cso_table_1'
response = requests.get(url)

response.status_code

200

In [12]:
page = response.text

In [13]:
soup = BeautifulSoup(page, 'lxml')

In [14]:
def movie_value(soup, field_name):
    
    obj = soup.find(text=re.compile(field_name))
    
    if not obj: 
        return None
    
    # this works for most of the values
    next_element = obj.findNext()
    
    if next_element:
        return next_element.text 
    else:
        return None

In [25]:
'''distributor_str = movie_value(soup, 'Distributor').split('See')[0]
distributor_str'''

"distributor_str = movie_value(soup, 'Distributor').split('See')[0]\ndistributor_str"

In [26]:
'''world_gross_str = soup.find(class_ = 'a-section a-spacing-none mojo-performance-summary-table').find_all(class_ = 'money')
worldwide_gross = world_gross_str[-1].text
worldwide_gross'''

"world_gross_str = soup.find(class_ = 'a-section a-spacing-none mojo-performance-summary-table').find_all(class_ = 'money')\nworldwide_gross = world_gross_str[-1].text\nworldwide_gross"

In [27]:
'''# rating
try:
    rating = movie_value(soup, 'MPAA')
except Exception:
    pass

print(rating)'''

"# rating\ntry:\n    rating = movie_value(soup, 'MPAA')\nexcept Exception:\n    pass\n\nprint(rating)"

Created function using selenium to scrape the lead actor/actress for all 1000 movies and put them into a list. This was added as a new column in my dataframe.

In [15]:
from bs4 import BeautifulSoup
import requests
import time, os

In [16]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

In [17]:
driver = webdriver.Chrome(chromedriver)

In [18]:
actor_list = []

try:
    for link in top_movies.link_stub:
        base_url = 'https://www.boxofficemojo.com'
        url = base_url + link
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        short_link = driver.find_element_by_xpath("//*[@id='tabs']/div/a[2]")
        short_link.click()
        time.sleep(1)
        actor = driver.find_element_by_xpath('//*[@id="principalCast"]/tbody/tr[2]/td[1]/a')
        actor = actor.text
        actor_list.append(actor)
except NoSuchElementException:
    pass

In [20]:
len(actor_list)

1000

In [21]:
top_movies['actor_actress'] = actor_list
top_movies.head()

,link_stub,rank,title,lifetime_gross,year,actor_actress
/title/tt2488496/?ref_=bo_cso_table_1,/title/tt2488496/?ref_=bo_cso_table_1,1,Star Wars: Episode VII - The Force Awakens,"$936,662,225",2015,Daisy Ridley
/title/tt4154796/?ref_=bo_cso_table_2,/title/tt4154796/?ref_=bo_cso_table_2,2,Avengers: Endgame,"$858,373,000",2019,Robert Downey Jr.
/title/tt0499549/?ref_=bo_cso_table_3,/title/tt0499549/?ref_=bo_cso_table_3,3,Avatar,"$760,507,625",2009,Sam Worthington
/title/tt1825683/?ref_=bo_cso_table_4,/title/tt1825683/?ref_=bo_cso_table_4,4,Black Panther,"$700,426,566",2018,Chadwick Boseman
/title/tt4154756/?ref_=bo_cso_table_5,/title/tt4154756/?ref_=bo_cso_table_5,5,Avengers: Infinity War,"$678,815,482",2018,Robert Downey Jr.


In [22]:
top_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, /title/tt2488496/?ref_=bo_cso_table_1 to /title/tt0075265/?ref_=bo_cso_table_200
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   link_stub       1000 non-null   object
 1   rank            1000 non-null   object
 2   title           1000 non-null   object
 3   lifetime_gross  1000 non-null   object
 4   year            1000 non-null   object
 5   actor_actress   1000 non-null   object
dtypes: object(6)
memory usage: 54.7+ KB


In [23]:
top_movies.to_csv('first_movies_df.csv', index=False)

## More functions
Created a function to scrape all info I wanted from each movie's page. This dictionary was converted into a dataframe and merged with the earlier dataframe.

In [24]:
def get_movie_dict(link):
    
    base_url = 'https://www.boxofficemojo.com'
    
    url = base_url + link
    
    #Request HTML and parse
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page,"lxml")
    
    movie_dict = {}

    #link_stub
    movie_dict['link_stub'] = link
    
    #title
    title_string = soup.find('title').text
    movie_dict['title'] = title_string.split(' - Box')[0].strip()
    
    # budget
    try:
        movie_dict['budget'] = movie_value(soup,'Budget')
    except AttributeError:
        pass
    
    #rating
    try:
        movie_dict['rating'] = movie_value(soup, 'MPAA')
    except AttributeError:
        pass
    
    # genre
    try:
        movie_dict['genre'] = movie_value(soup,'Genres').split()
    except AttributeError:
        pass
  
    #runtime
    raw_runtime = movie_value(soup,'Running')
    try:
        raw_runtime = raw_runtime.split()
        movie_dict['runtime (mins)'] = int(raw_runtime[0])*60 + int(raw_runtime[2])
    except:
        pass

    #release date
    try:
        release_date_str = soup.find(text = re.compile('Release Date'))
        movie_dict['release date'] = release_date_str.findNext().text.split('\n')[0]
    except AttributeError:
        pass
    
    #ditributor2
    try:
        movie_dict['distributor'] = movie_value(soup, 'Distributor').split('See')[0]
    except AttributeError:
        pass
    
    #opening gross
    try:
        movie_dict['opening gross'] = movie_value(soup, 'Opening')
    except AttributeError:
        pass
    
    return movie_dict

    

In [25]:
top_movies_list = []

for link in top_movies.link_stub:
    top_movies_list.append(get_movie_dict(link))
    time.sleep(2)

In [26]:
print(len(top_movies_list))

1000


In [27]:
#convert list to df
top_movies_df = pd.DataFrame(top_movies_list)  
top_movies_df.set_index('link_stub', inplace=True)

top_movies_df.shape

(1000, 8)

## EDA
Check for nulls and either replace or drop. Convert data types as needed. Create dummies for categorical features.

In [28]:
top_movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, /title/tt2488496/?ref_=bo_cso_table_1 to /title/tt0075265/?ref_=bo_cso_table_200
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           1000 non-null   object 
 1   budget          820 non-null    object 
 2   rating          863 non-null    object 
 3   genre           992 non-null    object 
 4   runtime (mins)  971 non-null    float64
 5   release date    992 non-null    object 
 6   distributor     991 non-null    object 
 7   opening gross   968 non-null    object 
dtypes: float64(1), object(7)
memory usage: 70.3+ KB


In [29]:
bool_series = pd.isnull(top_movies_df['rating'])

In [31]:
top_movies_df[bool_series].head()

,title,budget,rating,genre,runtime (mins),release date,distributor,opening gross
link_stub,,,,,,,,
/title/tt0383574/?ref_=bo_cso_table_27,Server Error,None,None,NaN,NaN,NaN,NaN,None
/title/tt0435761/?ref_=bo_cso_table_30,Toy Story 3,"$200,000,000",None,"[Adventure, Animation, Comedy, Family, Fantasy]",103.0,"June 16, 2010",Walt Disney Studios Motion Pictures,"$110,307,189"
/title/tt0266543/?ref_=bo_cso_table_43,Finding Nemo,"$94,000,000",None,"[Adventure, Animation, Comedy, Family]",100.0,"May 30, 2003",Walt Disney Studios Motion Pictures,"$70,251,710"
/title/tt0099785/?ref_=bo_cso_table_102,Home Alone,"$18,000,000",None,"[Comedy, Family]",103.0,"November 16, 1990",Twentieth Century Fox,"$17,081,997"
/title/tt0295297/?ref_=bo_cso_table_114,Server Error,None,None,NaN,NaN,NaN,NaN,None


In [32]:
top_movies.shape

(1000, 6)

In [130]:
top_movies_merge = top_movies.merge(top_movies_df, left_index=True, right_index=True)

top_movies_merge.shape
top_movies_merge.head()

,link_stub,rank,title_x,lifetime_gross,year,actor_actress,title_y,budget,rating,genre,runtime (mins),release date,distributor,opening gross
/title/tt2488496/?ref_=bo_cso_table_1,/title/tt2488496/?ref_=bo_cso_table_1,1,Star Wars: Episode VII - The Force Awakens,"$936,662,225",2015,Daisy Ridley,Star Wars: Episode VII - The Force Awakens,"$245,000,000",PG-13,"[Action, Adventure, Sci-Fi]",138.0,"December 16, 2015",Walt Disney Studios Motion Pictures,"$247,966,675"
/title/tt4154796/?ref_=bo_cso_table_2,/title/tt4154796/?ref_=bo_cso_table_2,2,Avengers: Endgame,"$858,373,000",2019,Robert Downey Jr.,Avengers: Endgame,"$356,000,000",PG-13,"[Action, Adventure, Drama, Sci-Fi]",181.0,"April 24, 2019",Walt Disney Studios Motion Pictures,"$357,115,007"
/title/tt0499549/?ref_=bo_cso_table_3,/title/tt0499549/?ref_=bo_cso_table_3,3,Avatar,"$760,507,625",2009,Sam Worthington,Avatar,"$237,000,000",PG-13,"[Action, Adventure, Fantasy, Sci-Fi]",162.0,"December 16, 2009",Twentieth Century Fox,"$77,025,481"
/title/tt1825683/?ref_=bo_cso_table_4,/title/tt1825683/?ref_=bo_cso_table_4,4,Black Panther,"$700,426,566",2018,Chadwick Boseman,Black Panther,None,PG-13,"[Action, Adventure, Sci-Fi]",134.0,"February 13, 2018",Walt Disney Studios Motion Pictures,"$202,003,951"
/title/tt4154756/?ref_=bo_cso_table_5,/title/tt4154756/?ref_=bo_cso_table_5,5,Avengers: Infinity War,"$678,815,482",2018,Robert Downey Jr.,Avengers: Infinity War,None,PG-13,"[Action, Adventure, Sci-Fi]",149.0,"April 25, 2018",Walt Disney Studios Motion Pictures,"$257,698,183"


In [131]:
#set(top_movies.index) - set(top_movies_df.index)

In [132]:
#set(top_movies_df.index)-set(top_movies.index)

In [133]:
#check for nulls
top_movies_merge.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, /title/tt2488496/?ref_=bo_cso_table_1 to /title/tt0075265/?ref_=bo_cso_table_200
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   link_stub       1000 non-null   object 
 1   rank            1000 non-null   object 
 2   title_x         1000 non-null   object 
 3   lifetime_gross  1000 non-null   object 
 4   year            1000 non-null   object 
 5   actor_actress   1000 non-null   object 
 6   title_y         1000 non-null   object 
 7   budget          820 non-null    object 
 8   rating          863 non-null    object 
 9   genre           992 non-null    object 
 10  runtime (mins)  971 non-null    float64
 11  release date    992 non-null    object 
 12  distributor     991 non-null    object 
 13  opening gross   968 non-null    object 
dtypes: float64(1), object(13)
memory usage: 149.5+ KB


In [134]:
top_movies_merge.to_csv('top_movies_merge_1.csv', index=False)

In [99]:
import pandas as pd
top_movies_merge = pd.read_csv('top_movies_merge_1.csv')

In [100]:
bool_series = pd.isnull(top_movies_merge['genre'])

In [101]:
top_movies_merge[bool_series]

,link_stub,rank,title_x,lifetime_gross,year,actor_actress,title_y,budget,rating,genre,runtime (mins),release date,distributor,opening gross
26,/title/tt0383574/?ref_=bo_cso_table_27,27,Pirates of the Caribbean: Dead Man's Chest,"$423,315,812",2006,Johnny Depp,Server Error,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,/title/tt0295297/?ref_=bo_cso_table_114,114,Harry Potter and the Chamber of Secrets,"$262,450,136",2002,Daniel Radcliffe,Server Error,NaN,NaN,NaN,NaN,NaN,NaN,NaN
148,/title/tt0407304/?ref_=bo_cso_table_149,149,War of the Worlds,"$234,280,354",2005,Tom Cruise,Server Error,NaN,NaN,NaN,NaN,NaN,NaN,NaN
295,/title/tt1403865/?ref_=bo_cso_table_96,296,True Grit,"$171,243,005",2010,Jeff Bridges,Server Error,NaN,NaN,NaN,NaN,NaN,NaN,NaN
342,/title/tt0106918/?ref_=bo_cso_table_143,343,The Firm,"$158,348,367",1993,Tom Cruise,Server Error,NaN,NaN,NaN,NaN,NaN,NaN,NaN
437,/title/tt0130623/?ref_=bo_cso_table_38,438,Dinosaur,"$137,748,063",2000,D.B. Sweeney,Server Error,NaN,NaN,NaN,NaN,NaN,NaN,NaN
456,/title/tt2024432/?ref_=bo_cso_table_57,457,Identity Thief,"$134,506,920",2013,Jason Bateman,Server Error,NaN,NaN,NaN,NaN,NaN,NaN,NaN
707,/title/tt1568346/?ref_=bo_cso_table_108,708,The Girl with the Dragon Tattoo,"$102,515,793",2011,Daniel Craig,Server Error,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
print(len(top_movies_merge[bool_series]))

8


In [103]:
#turn year into int
top_movies_merge['year'] = top_movies_merge['year'].astype(int)

In [104]:
top_movies_merge['year']

0      2015
1      2019
2      2009
3      2018
4      2018
       ... 
995    2008
996    1984
997    2010
998    2019
999    1976
Name: year, Length: 1000, dtype: int64

In [105]:
#handle runtime nulls
top_movies_merge['runtime (mins)'] = top_movies_merge['runtime (mins)'].fillna(113.0)

top_movies_merge['runtime (mins)']

0      138.0
1      181.0
2      162.0
3      134.0
4      149.0
       ...  
995     85.0
996    107.0
997    108.0
998    114.0
999    139.0
Name: runtime (mins), Length: 1000, dtype: float64

In [106]:
#handle budget nulls
top_movies_merge['budget'] = top_movies_merge['budget'].astype(str)

top_movies_merge["budget"] = top_movies_merge["budget"].str.replace(",","")

In [107]:
top_movies_merge['budget'] = top_movies_merge['budget'].str.replace('$',"")

<ipython-input-107-5159ae65845a>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  top_movies_merge['budget'] = top_movies_merge['budget'].str.replace('$',"")


In [108]:
#top_movies_merge['new budget'] = top_movies_merge['budget'].dropna()

#top_movies_merge['new budget'] = top_movies_merge['new budget'].replace('None', '0')

#top_movies_merge['new budget'] = top_movies_merge['new budget'].astype(float)

#top_movies_merge['new budget'].mean()

In [109]:
top_movies_merge['budget'] = top_movies_merge.budget.replace('nan','72067774')

In [110]:
top_movies_merge['budget'].head()

0    245000000
1    356000000
2    237000000
3     72067774
4     72067774
Name: budget, dtype: object

In [111]:
top_movies_merge['budget'] = top_movies_merge['budget'].astype(int)

top_movies_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   link_stub       1000 non-null   object 
 1   rank            1000 non-null   object 
 2   title_x         1000 non-null   object 
 3   lifetime_gross  1000 non-null   object 
 4   year            1000 non-null   int64  
 5   actor_actress   1000 non-null   object 
 6   title_y         1000 non-null   object 
 7   budget          1000 non-null   int64  
 8   rating          863 non-null    object 
 9   genre           992 non-null    object 
 10  runtime (mins)  1000 non-null   float64
 11  release date    992 non-null    object 
 12  distributor     991 non-null    object 
 13  opening gross   968 non-null    object 
dtypes: float64(1), int64(2), object(11)
memory usage: 109.5+ KB


In [112]:
#top_movies_merge['new open gross'] = top_movies_merge['opening gross'].dropna() 

In [113]:
#top_movies_merge['new open gross'] = top_movies_merge['new open gross'].astype(str)

#top_movies_merge['new open gross'] = top_movies_merge['new open gross'].str.replace(",","")

In [114]:
#top_movies_merge['new open gross'] = top_movies_merge['new open gross'].str.replace('$',"")

In [115]:
#top_movies_merge['new open gross'] = top_movies_merge['new open gross'].replace('nan', '0')

In [116]:
#top_movies_merge['new open gross'] = top_movies_merge['new open gross'].replace('Gross', '0')

In [117]:
#top_movies_merge['new open gross'] = top_movies_merge['new open gross'].astype(int)

In [118]:
#top_movies_merge['new open gross'].median()

In [119]:
#handle opening gross nulls
top_movies_merge['opening gross'] = top_movies_merge['opening gross'].astype(str)

top_movies_merge['opening gross'] = top_movies_merge['opening gross'].str.replace(",","")

In [120]:
top_movies_merge['opening gross'] = top_movies_merge['opening gross'].str.replace('$',"")

top_movies_merge['opening gross'] = top_movies_merge['opening gross'].replace('nan','31113954')

top_movies_merge['opening gross'] = top_movies_merge['opening gross'].replace('Gross','31113954')

<ipython-input-120-bdc1ffd094b7>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  top_movies_merge['opening gross'] = top_movies_merge['opening gross'].str.replace('$',"")


In [121]:
top_movies_merge['opening gross'] = top_movies_merge['opening gross'].astype(int)

top_movies_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   link_stub       1000 non-null   object 
 1   rank            1000 non-null   object 
 2   title_x         1000 non-null   object 
 3   lifetime_gross  1000 non-null   object 
 4   year            1000 non-null   int64  
 5   actor_actress   1000 non-null   object 
 6   title_y         1000 non-null   object 
 7   budget          1000 non-null   int64  
 8   rating          863 non-null    object 
 9   genre           992 non-null    object 
 10  runtime (mins)  1000 non-null   float64
 11  release date    992 non-null    object 
 12  distributor     991 non-null    object 
 13  opening gross   1000 non-null   int64  
dtypes: float64(1), int64(3), object(10)
memory usage: 109.5+ KB


In [122]:
#top_movies_merge = top_movies_merge.drop(['new open gross','new budget'], axis=1)

In [123]:
#top_movies_merge.info()

In [124]:
#create dummies for actor_actress, distributor
top_movies_merge = pd.get_dummies(top_movies_merge, columns = ['distributor'], drop_first=True)

In [125]:
top_movies_merge.head()

,link_stub,rank,title_x,lifetime_gross,year,actor_actress,title_y,budget,rating,genre,...,distributor_Summit Entertainment,distributor_The Weinstein Company,distributor_TriStar Pictures,distributor_Twentieth Century Fox,distributor_USA Films,distributor_United Artists,distributor_United Artists Releasing,distributor_Universal Pictures,distributor_Walt Disney Studios Motion Pictures,distributor_Warner Bros.
0,/title/tt2488496/?ref_=bo_cso_table_1,1,Star Wars: Episode VII - The Force Awakens,"$936,662,225",2015,Daisy Ridley,Star Wars: Episode VII - The Force Awakens,245000000,PG-13,"['Action', 'Adventure', 'Sci-Fi']",...,0,0,0,0,0,0,0,0,1,0
1,/title/tt4154796/?ref_=bo_cso_table_2,2,Avengers: Endgame,"$858,373,000",2019,Robert Downey Jr.,Avengers: Endgame,356000000,PG-13,"['Action', 'Adventure', 'Drama', 'Sci-Fi']",...,0,0,0,0,0,0,0,0,1,0
2,/title/tt0499549/?ref_=bo_cso_table_3,3,Avatar,"$760,507,625",2009,Sam Worthington,Avatar,237000000,PG-13,"['Action', 'Adventure', 'Fantasy', 'Sci-Fi']",...,0,0,0,1,0,0,0,0,0,0
3,/title/tt1825683/?ref_=bo_cso_table_4,4,Black Panther,"$700,426,566",2018,Chadwick Boseman,Black Panther,72067774,PG-13,"['Action', 'Adventure', 'Sci-Fi']",...,0,0,0,0,0,0,0,0,1,0
4,/title/tt4154756/?ref_=bo_cso_table_5,5,Avengers: Infinity War,"$678,815,482",2018,Robert Downey Jr.,Avengers: Infinity War,72067774,PG-13,"['Action', 'Adventure', 'Sci-Fi']",...,0,0,0,0,0,0,0,0,1,0


In [126]:
#top_movies_merge = pd.get_dummies(top_movies_merge, columns = ['actor_actress'], drop_first=True)

In [127]:
#top_movies_merge.head()

In [60]:
gen = pd.get_dummies(top_movies_merge['genre'].apply(pd.Series).stack())

In [61]:
gen_df = pd.DataFrame(gen)

In [62]:
gen_df

,,"['Action', 'Adventure', 'Animation', 'Comedy', 'Family', 'Fantasy', 'Musical']","['Action', 'Adventure', 'Animation', 'Comedy', 'Family', 'Fantasy', 'Sci-Fi']","['Action', 'Adventure', 'Animation', 'Comedy', 'Family', 'Fantasy']","['Action', 'Adventure', 'Animation', 'Comedy', 'Family', 'Sci-Fi', 'Thriller']","['Action', 'Adventure', 'Animation', 'Comedy', 'Family', 'Sci-Fi']","['Action', 'Adventure', 'Animation', 'Comedy', 'Family']","['Action', 'Adventure', 'Animation', 'Family', 'Fantasy', 'Sci-Fi', 'Sport']","['Action', 'Adventure', 'Animation', 'Family', 'Fantasy', 'Sci-Fi']","['Action', 'Adventure', 'Animation', 'Family', 'Fantasy']","['Action', 'Adventure', 'Animation', 'Family', 'Sci-Fi']",...,"['Fantasy', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller']","['Horror', 'Mystery', 'Thriller']","['Horror', 'Mystery']","['Horror', 'Sci-Fi', 'Thriller']","['Horror', 'Sci-Fi']","['Horror', 'Thriller']",['Horror'],"['Musical', 'Romance']","['Mystery', 'Sci-Fi', 'Thriller']","['Mystery', 'Thriller']"
0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
final_df = top_movies_merge.merge(gen_df, left_index=True, right_index=True)

final_df.shape

(1000, 386)

In [40]:
final_df = pd.get_dummies(final_df, columns = ['rating'], drop_first=True)

In [41]:
'''#create covid
for date_str in final_df['release date']:
    if date_str < 'March 15, 2020':
        final_df['pre-covid'].append(date_str)
    else:
        final_df['covid'].append(date_str)'''

"#create covid\nfor date_str in final_df['release date']:\n    if date_str < 'March 15, 2020':\n        final_df['pre-covid'].append(date_str)\n    else:\n        final_df['covid'].append(date_str)"

In [42]:
#try to fix rating so that not as many drop
final_df = final_df.dropna()

In [43]:
final_df.corr()

,year,budget,runtime (mins),opening gross,distributor_American International Pictures (AIP),distributor_Artisan Entertainment,distributor_Columbia Pictures,distributor_Dimension Films,distributor_DreamWorks,distributor_DreamWorks Distribution,...,"['Horror', 'Sci-Fi']","['Horror', 'Thriller']",['Horror'],"['Musical', 'Romance']","['Mystery', 'Sci-Fi', 'Thriller']","['Mystery', 'Thriller']",rating_G,rating_PG,rating_PG-13,rating_R
year,1.000000,0.311140,0.026509,0.376008,-0.065746,-0.013589,-0.182379,-0.022220,0.055279,-0.032892,...,-0.065746,0.033353,-0.072925,-0.068353,0.017705,-0.009996,-0.171193,0.107874,0.273336,0.067629
budget,0.311140,1.000000,0.269998,0.565785,-0.007451,-0.048480,-0.071824,-0.075391,0.113449,-0.032238,...,-0.042246,-0.043386,-0.052075,-0.045095,-0.020025,0.007124,-0.006913,0.081252,0.275500,-0.264471
runtime (mins),0.026509,0.269998,1.000000,0.215143,0.002291,-0.052082,0.005380,-0.067212,-0.077341,-0.025559,...,0.002291,0.002291,0.030305,-0.008281,-0.005260,0.058965,-0.158274,-0.292352,0.239773,0.130077
opening gross,0.376008,0.565785,0.215143,1.000000,-0.029114,-0.034748,-0.083575,-0.023888,0.045424,-0.031651,...,-0.032954,-0.000491,0.013648,-0.028137,-0.004549,0.004199,-0.015172,0.004525,0.296866,-0.148457
distributor_American International Pictures (AIP),-0.065746,-0.007451,0.002291,-0.029114,1.000000,-0.001009,-0.003801,-0.002678,-0.004784,-0.004195,...,-0.001009,-0.001009,0.499243,-0.001009,-0.001009,-0.001750,-0.005211,-0.016561,-0.026607,-0.016710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"['Mystery', 'Thriller']",-0.009996,0.007124,0.058965,0.004199,-0.001750,-0.001750,-0.006590,-0.004643,-0.008294,-0.007273,...,-0.001750,-0.001750,-0.003504,-0.001750,-0.001750,1.000000,-0.009036,-0.028713,-0.008835,0.015586
rating_G,-0.171193,-0.006913,-0.158274,-0.015172,-0.005211,-0.005211,-0.019629,-0.013830,-0.024707,-0.021663,...,-0.005211,-0.005211,-0.010439,-0.005211,-0.005211,-0.009036,1.000000,-0.085530,-0.137412,-0.086299
rating_PG,0.107874,0.081252,-0.292352,0.004525,-0.016561,-0.016561,-0.041528,0.014808,0.138578,0.044849,...,-0.016561,-0.016561,-0.033172,0.060932,-0.016561,-0.028713,-0.085530,1.000000,-0.436665,-0.274239
rating_PG-13,0.273336,0.275500,0.239773,0.296866,-0.026607,-0.026607,-0.100213,-0.021678,-0.014888,-0.031702,...,-0.026607,0.037926,-0.053294,-0.026607,0.037926,-0.008835,-0.137412,-0.436665,1.000000,-0.440591


In [44]:
final_df.shape

(992, 389)

In [45]:
final_df.to_csv('complete_web_scraping.csv', index=False)